In [ ]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
import numpy as np
import keras
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.layers import Dense,Reshape,Input,BatchNormalization,Dropout,Concatenate,Flatten,Lambda,Resizing,CuDNNLSTM,concatenate,Layer
from keras import  Model,layers
from tensorflow.keras.applications import VGG16 
import tensorflow as tf

In [ ]:
input_shape= (25,40, 3)
batch_size = 64

In [ ]:
class MULTILSTM(Layer):
  def __init__(self, size,num, return_sequences, **kwargs):
     super(MULTILSTM, self).__init__()
     self.size = size
     self.num = num
     self.return_sequences = return_sequences
     self.layers = [CuDNNLSTM(self.size,return_sequences=self.return_sequences) for i in range(self.num)]

  def call(self,inputs):
      outs = []
      for axis in range(inputs.shape[2]):
          out = self.layers[axis](inputs[:,:,axis,:])
          if self.return_sequences:
             out = keras.backend.expand_dims(out,axis=2)
          outs.append(out)
      return concatenate(outs,axis=2) if self.return_sequences else concatenate(outs)
   
  def get_config(self):
          config = super().get_config().copy()
          config.update({
             'size' : self.size,
             'layers' : self.layers,
             'num': self.num,
             'return_sequences': self.return_sequences
          })
          return config

def lstm(inputs,axis):
   x = inputs[:,axis,:]
   x = CuDNNLSTM(100,return_sequences=False)(x)
   return x  

In [ ]:
base_path = '/content/drive/MyDrive/OAD/OAD AUG/'
train_x = np.load(base_path+'train_x.npy')
test_x = np.load(base_path+'test_x.npy')
train_y = np.load(base_path+'train_y.npy')
test_y = np.load(base_path+'test_y.npy')

In [ ]:
#####
train_data = tf.data.Dataset.from_tensor_slices((train_x,train_y))
val_data = tf.data.Dataset.from_tensor_slices((test_x,test_y))
train_data = train_data.batch(batch_size)
val_data = val_data.batch(batch_size)
#####
options = tf.data.Options()
options.experimental_distribute.auto_shard_policy = tf.data.experimental.AutoShardPolicy.DATA
train_data = train_data.with_options(options)
val_data = val_data.with_options(options)

In [ ]:
gpus = tf.config.list_logical_devices('GPU')
strategy = tf.distribute.MirroredStrategy(gpus)
with strategy.scope():
    inp=Input(np.asarray(input_shape))
    output = Resizing(224, 224)(inp)
    model = VGG16(include_top=False, input_shape=(224,224,3))
    for layer in model.layers:
        layer.trainable = True
    output = model(output)
    output = Flatten()(output)
    output = Dense(4096)(output)
    output = BatchNormalization()(output)
    output = Dropout(0.5)(output)
    output = Dense(4096)(output)
    output = BatchNormalization()(output)
    output = Dropout(0.5)(output)
    output = Dense(11, activation='softmax',kernel_regularizer='l1')(output)
    model = Model(inp, output)
    model.summary()
    model.compile(loss=keras.losses.categorical_crossentropy,
                  optimizer='adam',
                  metrics=['accuracy'])

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0',)
58900480/58889256 [==============================] - 0s 0us/step
Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 25, 40, 3)]       0         
                                                                 
 resizing (Resizing)         (None, 224, 224, 3)       0         
                                                                 
 vgg16 (Functional)          (None, 7, 7, 512)         14714688  
                                                                 
 flatten (Flatten)           (None, 25088)             0         
                                                                 
 dense (Dense)               (None, 8192)              205529088 
                                                                 
 dense_1 (Dense)          

In [ ]:
filepath ="oad.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=0, save_best_only=True, mode='min')
callbacks = [checkpoint,EarlyStopping(monitor='val_loss', patience=100,)]

In [ ]:
history=model.fit(train_x,
                  train_y,
                  epochs=50,
                  batch_size=batch_size,
                  validation_split=0.25,
                  verbose=1,
                  callbacks=callbacks)
model.load_weights(filepath)
keras.models.save_model(model, '/content/drive/MyDrive/VGG16_OAD.h5')
print(model.evaluate(test_x,test_y))

Epoch 1/50
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
851/851 [==============================] - 149s 160ms/step - loss: 31.4594 - accuracy: 0.8235 - val_loss: 4.6613 - val_accuracy: 0.8458
Epoch 2/50
85